In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
import time
from IPython.display import clear_output

# Real Time Anomaly Detection
Fungsi untuk mendeteksi anomali secara real-time

In [ ]:
def real_time_anomaly_detection(model_path, scaler_path, data_path, threshold, interval=5, max_iterations=100):
    # Memuat model dan scaler
    model = load_model(model_path, compile=False)
    scaler = joblib.load(scaler_path)

    # Mengidentifikasi fitur yang digunakan model
    features = [col for col in pd.read_csv(data_path).columns
                if col not in ['id', 'timestamp', 'reconstruction_error', 'is_anomaly']]

    # Inisialisasi dataframe untuk melacak anomali
    anomaly_tracking = pd.DataFrame(columns=['timestamp', 'anomaly_count', 'total_samples', 'anomaly_percentage'])

    # Mulai deteksi anomali real-time
    print("Memulai deteksi anomali real-time...")
    for i in range(max_iterations):
        # Timestamp saat ini
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

        # Memuat data terbaru
        df = pd.read_csv(data_path)

        # Mendeteksi anomali
        X = df[features]
        X_scaled = scaler.transform(X)
        X_pred = model.predict(X_scaled)
        mse = np.mean(np.power(X_scaled - X_pred, 2), axis=1)

        # Menambahkan hasil ke dataframe
        df['reconstruction_error'] = mse
        df['is_anomaly'] = mse > threshold

        # Menghitung jumlah anomali dan persentase
        anomaly_count = df['is_anomaly'].sum()
        total_samples = len(df)
        anomaly_percentage = (anomaly_count / total_samples) * 100

        # Mencatat statistik anomali
        anomaly_tracking = pd.concat([
            anomaly_tracking,
            pd.DataFrame({
                'timestamp': [timestamp],
                'anomaly_count': [anomaly_count],
                'total_samples': [total_samples],
                'anomaly_percentage': [anomaly_percentage]
            })
        ], ignore_index=True)

        # Menampilkan hasil
        clear_output(wait=True)
        print(f"Iterasi {i+1}/{max_iterations} - {timestamp}")
        print(f"Total sampel: {total_samples}")
        print(f"Anomali terdeteksi: {anomaly_count} ({anomaly_percentage:.2f}%)")

        # Menampilkan sampel anomali terbaru
        if anomaly_count > 0:
            print("\nSampel anomali terbaru:")
            anomalies = df[df['is_anomaly'] == True].sort_values('reconstruction_error', ascending=False).head(5)
            display(anomalies[['id', 'created_date', 'created_time', 'reconstruction_error']])

        # Visualisasi tren anomali
        plt.figure(figsize=(12, 6))
        plt.plot(anomaly_tracking['timestamp'], anomaly_tracking['anomaly_percentage'], marker='o')
        plt.title('Tren Persentase Anomali')
        plt.xlabel('Waktu')
        plt.ylabel('Persentase Anomali (%)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Visualisasi distribusi error rekonstruksi
        plt.figure(figsize=(12, 6))
        sns.histplot(df['reconstruction_error'], bins=30)
        plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.6f}')
        plt.title('Distribusi Reconstruction Error')
        plt.xlabel('Reconstruction Error (MSE)')
        plt.ylabel('Jumlah Sampel')
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Jika bukan iterasi terakhir, tunggu sebelum iterasi berikutnya
        if i < max_iterations - 1:
            print(f"\nMenunggu {interval} detik untuk pengambilan data berikutnya...")
            time.sleep(interval)

    print("Deteksi anomali real-time selesai.")
    return anomaly_tracking

In [ ]:
anomaly_stats = real_time_anomaly_detection(
    model_path='anomaly_autoencoder_model.h5',
    scaler_path='anomaly_scaler.pkl',
    data_path='tb_sig_5.csv',
    threshold=0.01,  # Sesuaikan dengan nilai threshold yang diperoleh dari model
    interval=10,
    max_iterations=20
)